Prompts

PromptTemplate — Criação de Prompts com Variáveis Dinâmicas
Objeto:

PromptTemplate é um modelo reutilizável de texto com espaços reservados (placeholders) para valores dinâmicos.

In [ ]:
from langchain.prompts.prompt import PromptTemplate

TEMPLATE = """
Você é um assistente que traduz do {input_language} para o {output_language}.
"""

# Criando o prompt
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input_language", "output_language"])

# Formatando com valores reais
prompt_template.format(input_language="inglês", output_language="alemão")


Few-Shot Prompting — Exemplos dentro do Prompt
 Objeto:

PromptTemplate com exemplos embutidos no texto. Útil para mostrar ao modelo como você quer que ele pense ou responda

Vantagens:

    Melhora a qualidade e consistência da resposta

    Mostra ao modelo o formato esperado

    Muito usado em NLP tradicional + LLMs

In [ ]:
TEMPLATE = """
Interprete o texto e classifique:
- sentimento (positivo, neutro, negativo)
- assunto (uma palavra)

Formato JSON:
sentiment
subject

Texto: {input}

Exemplos:
Texto: A loja esportiva Centauro oferece uma experiência maravilhosa na compra de material esportivo em geral
sentiment: positivo
subject: Centauro

Texto: Centauro tive um atendimento mediano. Os produtos são ok...
sentiment: neutro
subject: Centauro

Texto: Centauro tem um produto horrivel. E o atendimento de saq foi horrivel...
sentiment: negativo
subject: Centauro


"""

prompt = PromptTemplate(template=TEMPLATE, input_variables=["input"])


 3. **FewShotPromptTemplate** — Modularização de Exemplos
 Objeto:

**FewShotPromptTemplate** permite criar prompts com exemplos estruturados, separados por **chave** e **valor**.

Vantagens:

    Separação clara entre exemplos e estrutura

    Mais flexível e escalável

    Facilita adicionar/remover exemplos

from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
    {"text": "O atendimento foi ótimo!", "response": "sentiment: positivo\nsubject: atendimento"},
    {"text": "O produto estava danificado.", "response": "sentiment: negativo\nsubject: loja"}
]

example_prompt = PromptTemplate(
    input_variables=["text", "response"],
    template="Texto: {text}\n{response}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Texto: {input}",
    input_variables=["input"]
)

print(prompt.format(input="O camisa que comprei veio com defeito na manga da camisa."))


 **Chain-of-Thought Prompting** — Explicações no Prompt
 Objeto:
 Ainda é um PromptTemplate, mas agora com explicações passo a passo, mostrando como o modelo deve pensar antes de responder.

 Vantagens:

    Leva o modelo a "raciocinar"

    Aumenta precisão em tarefas complexas

    Reduz erros por falta de contexto

In [ ]:
TEMPLATE = """
Texto: {input}

Vamos pensar juntos:
"O produto estava danificado, e o atendimento foi ruim." O que isso nos diz?
É uma crítica negativa.

Resposta em JSON:
{ "sentiment": "negativo", "subject": "loja" }
"""


**PipelinePromptTemplate** — Composição de Prompts
 Objeto:

**PipelinePromptTemplate** permite juntar vários templates pequenos para montar um prompt maior, organizado por seções (ex: introdução, exemplo, execução).

**Vantagens**:

    Permite reutilizar blocos de texto

    Ideal para projetos mais complexos e dinâmicos

    Deixa o código modular e limpo

from langchain.prompts.pipeline import PipelinePromptTemplate

# Define partes do prompt
introduction = PromptTemplate.from_template("Você vai analisar sentimentos em textos.")
example = PromptTemplate.from_template("Texto: {example_text}\nResposta: {example_response}")
execution = PromptTemplate.from_template("Agora analise o seguinte: {input}")

full_template = "{introduction}\n\n{example}\n\n{execution}"
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=PromptTemplate.from_template(full_template),
    pipeline_prompts=[
        ("introduction", introduction),
        ("example", example),
        ("execution", execution)
    ]
)

print(pipeline_prompt.format(
    example_text="O restaurante estava excelente!",
    example_response='{ "sentiment": "positivo", "subject": "restaurante" }',
    input="O prato estava sem gosto e demorou muito."
))


**Serialização de Prompts — save() e load_prompt()**
Objeto:

Qualquer PromptTemplate pode ser salvo em disco como .json ou .yaml, e recarregado depois.

**Vantagens:**

    Reutilização de prompts em diferentes projetos

    Permite versionamento de templates

    Ideal para ambientes de produção

In [ ]:
prompt = PromptTemplate(input_variables=["input"], template="Conte uma piada sobre {input}")
prompt.save("prompt.yaml")

from langchain.prompts import load_prompt
loaded_prompt = load_prompt("prompt.yaml")
print(loaded_prompt.format(input="galinhas"))
